In [2]:
#importing libraries
import math
import re
from nltk.corpus import stopwords

In [3]:
def create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("nepali"))

    for sent in sentences:
        freq_table = {}
        words=sent.split()
        for word in words:
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:10]] = freq_table

    return frequency_matrix

def create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix


def create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table


def create_idf_matrix(freq_matrix, count_doc_per_words, documents_size):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(documents_size / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix



def create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix



def generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = []

    for sentence in sentences:
        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] >= (threshold):
            summary.append(sentence)
            sentence_count += 1

    return summary


In [4]:

import anvil.server

anvil.server.connect("MC2OMWB6YG65OU74ZZBCLF7X-JFP3EUIT4WZLMX3X")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [16]:
import anvil.media


@anvil.server.callable
def summarize_text(input_text_lbl1):
        text = input_text_lbl1
        sents=re.split('।',text)
        documents_size = len(sents)
        words=text.split()
        freq_matrix = create_frequency_matrix(sents)
        tf_matrix = create_tf_matrix(freq_matrix)
        count_doc_per_words = create_documents_per_words(freq_matrix)
        idf_matrix = create_idf_matrix(freq_matrix,count_doc_per_words, documents_size)
        tf_idf_matrix = create_tf_idf_matrix(tf_matrix, idf_matrix)
        def sentence_scores(tf_idf_matrix) -> dict:


            sentenceValue = {}

            for sent, f_table in tf_idf_matrix.items():
                total_score_per_sentence = 0

                count_words_in_sentence = len(f_table)
                for word, score in f_table.items():
                    total_score_per_sentence += score
                if count_words_in_sentence !=0:
                    sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence
                else:
                    sentenceValue[sent]=0
            return sentenceValue



        def find_average_score(sentenceValue) -> int:
            sumValues = 0
            for entry in sentenceValue:
                sumValues += sentenceValue[entry]

            # Average value of a sentence from original summary_text
            average = (sumValues / len(sentenceValue))

            return average
        sentence_scores = sentence_scores(tf_idf_matrix)
        threshold = find_average_score(sentence_scores)
        print(text)
        summary = '।'.join(generate_summary(sents, sentence_scores,0.8*threshold ))
        return summary

१३ कात्तिक, काठमाडौं । अमेरिकी विदेशमन्त्री माइक पम्पियोले एक महिनाको अवधीमा दुई पटक एसियाको भ्रमण गरे ।

केही हप्ता पहिले हुने जापानमा भएको क्वाड बैठकमा सहभागी हुन आएका थिए र त्यही बेला उनले विभिन्न एसियाली मुलुकको भ्रमण गर्ने तय भएको थियो तर पछि रद्ध भयो ।




यतिबेला उनी दक्षिण एसिया र दक्षिणपूर्वी एसियाली मुलुकको भ्रमणमा छन् । उनले भारत, श्रीलंका, माल्दिभ्सको भ्रमण सम्पन्न गरेका छन् । उनले अन्तिम समयमा आएर अर्काे एसियाली मुलुक भियतनामलाई समेत आफ्नो भ्रमणको सूचिमा राखेका छन् ।



तर पम्पियो किन नेपाल आएनन् ? सन् २००२ कोलिन पावेल नेपालको भ्रमणमा आएदेखि कुनै पनि अमेरिकी विदेशमन्त्रीले नेपालको भ्रमण गरेका छैनन् । नेपालको तर्फबाट भने सन् २०१८ को डिसेम्बरमा परराष्ट्रमन्त्री प्रदीप ज्ञवालीले अमेरिकाको भ्रमण गरेका थिए । दक्षिण एसियाका साना मुलुकहरु भ्रमणको प्राथमिकतामा पर्दा नेपाल किन परेन ?

यस विषयमा सरकारी पक्षले केही पनि बताएको छैन । पूर्वप्रधानमन्त्री डा. बाबुराम भट्टराईले एक ट्वीट गर्दै भनेका छन्, ‘अहिले देशमा को आयो र गयो भन्ने रोइलो चलिरहेको बेला मचाँहि हाम्रो देश विश्व, क्षेत्रिय 